In [ ]:
import tensorflow as tf
from typing import Dict, Tuple
from google.colab import drive
import numpy as np
import pickle
import os

In [ ]:
# Load the data from Google Drive
def load_data_from_drive(folder_name: str) -> Tuple[np.ndarray, np.ndarray, Dict[int, str]]:
    drive.mount('/content/drive')
    folder_path = os.path.join('/content/drive/My Drive', folder_name)
    images = np.load(os.path.join(folder_path, 'images.npy'))
    labels = np.load(os.path.join(folder_path, 'labels.npy'))
    with open(os.path.join(folder_path, 'class_mapping.pkl'), 'rb') as f:
        class_dict = pickle.load(f)
    print(f"Loaded {len(images)} images and labels from {folder_name} in Google Drive")
    return images, labels, class_dict

# Create the Keras model for fine-tuning
def create_model_for_finetuning(num_classes: int) -> tf.keras.Model:
    base_model = tf.keras.applications.convnext.ConvNeXtLarge(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

In [ ]:
images, labels, class_dict = load_data_from_drive('michael_tanmay_ibm_ai_workshop')
print(images.shape)
print(labels.shape)

In [ ]:
images = tf.keras.applications.convnext.preprocess_input(images)

In [ ]:
num_classes = len(class_dict)
model = create_model_for_finetuning(num_classes=num_classes)
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = 'sparse_categorical_crossentropy'
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
import random
i = list(range(len(images)))
random.shuffle(i)
images = images[i]
labels = labels[i]

In [ ]:
batch_size = 32
shuffle = True
validation_split = 0.1
epochs = 2
model.fit(images, labels, batch_size=batch_size, shuffle=shuffle, validation_split=validation_split, epochs=epochs)

In [ ]:
from PIL import Image
from io import BytesIO
from google.colab import files

def load_image_from_upload() -> Image.Image:
    """
    Let the user upload an image into the Colab cell interface and load it using Pillow.

    Returns:
        A Pillow Image object of the uploaded image.
    """
    uploaded = files.upload()
    img_bytes = BytesIO(uploaded[next(iter(uploaded))])
    img = Image.open(img_bytes)
    return img

In [ ]:
image = load_image_from_upload().convert("RGB").resize((224, 224))

In [ ]:
image = tf.keras.applications.convnext.preprocess_input(np.array(image)[None, ...])
pred = model.predict(image)[0]

In [ ]:
best = pred.argsort()[-5:]
names = [class_dict[x] for x in best]
scores = pred[best]

for i in zip(best, names, scores):
    print(i)